# 1. 量化示例pytorch ptq & qat

#### 1.1 定义简单的浮点模块

In [1]:
import torch
from torch import nn, Tensor


class M(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = torch.nn.Conv2d(1, 1, 1)
        self.relu = torch.nn.ReLU()
        
    def _forward_impl(self, x:Tensor) -> Tensor:
        """提供便捷函数"""
        x = self.conv(x)
        x = self.relu(x)
        return x
    
    def forward(self, x:Tensor) -> Tensor:
        x = self._forward_impl(x)
        return x

#### 1.2 定义可量化模型

1)  将浮点模块‘M’转换为可量化模块‘QM’，（量化流程的最关键的一步）

In [2]:
from torch.ao.quantization import QuantStub, DeQuantStub

class QM(M):
    """_summary_

    Args:
        M (_type_): _description_
        is_print: 为了测试需求，打印一些信息
    """
    def __init__(self, is_print: bool=False):
        super().__init__()
        self.is_print = is_print
        self.quant = QuantStub()  # 将张量从浮点转换为量化
        self.dequant = DeQuantStub()  # 将张量从量化转换为浮点
        
    def forward(self, x:Tensor) -> Tensor:
        # 手动指定张量将在量化模型中从浮点模块转换为量化模块的位置；
        x = self.quant(x)
        if self.is_print:
            print("量化前的类型：", x.dtype)
        x = self._forward_impl(x)
        if self.is_print:
            print("量化中的类型：", x.dtype)
        # 在量化模型中手动指定张量从量化到浮点的转换位置；
        x = self.dequant(x)
        if self.is_print:
            print("量化后的类型：", x.dtype)
        return x

2)  简单测试前向过程中的激活数据类型；

In [3]:
input_fp32 = torch.randn(4, 1, 4, 4)  # 输入的数据

m = QM(is_print=True)
x = m(input_fp32)

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32


3)  查看权重的数据类型

In [4]:
m.conv.weight.dtype

torch.float32

# 2. PTQ 

当内存带宽和计算空间都很重要时，通常会使用训练后量化，而CNN就是其典型的用例。训练后量化对模型的权重和激活进行量化。他在可能的情况下将激活融合到前面的层中。他需要用具有代表性的数据集进行校准，以确定激活的最佳量化参数。

#### 2.1 静态量化模型

In [5]:
# 创建浮点模型实例
model_fp32 = QM(is_print=True)

In [6]:
# 使PTQ生效，将模型设置为eval模式
model_fp32.eval()

QM(
  (conv): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (quant): QuantStub()
  (dequant): DeQuantStub()
)

In [7]:
# 查看当前数据类型
input_fp32 = torch.randn(4, 1, 4, 4)

x = model_fp32(input_fp32)
print('激活和权重的数据类型分别为：'f'{x.dtype}, {model_fp32.conv.weight.dtype}')

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32
激活和权重的数据类型分别为：torch.float32, torch.float32


1)  定义观测器

复制实例变量'qconfig',其中包含关于要附加哪种观测器的信息：

-使用['fbgemm']用于带AVX2的x86; 使用['qnnpack']用于ARM CPU（通常出现在移动/嵌入式设备中)

-其他量化配置，如选择对称或非对称量化和'MinMax'或'L2Norm'校准技术，可以在这里指定

In [9]:
model_fp32.qconfig = torch.ao.quantization.get_default_qconfig('fbjemm')

In [10]:
# 查看此时的数据类型
input_fp32 = torch.randn(4, 1, 4, 4)

x = model_fp32(input_fp32)
print('激活和权重的数据类型分别为：'f'{x.dtype}, {model_fp32.conv.weight.dtype}')

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32
激活和权重的数据类型分别为：torch.float32, torch.float32


2)  融合激活层

在适用的地方，融合activation到前面的层（这需要根据模型架构手动完成）。常见的融合包括'conv+relu'和'conv+batchnorm+relu'.

In [11]:
model_fp32_fused = torch.ao.quantization.fuse_modules(model_fp32, [['conv', 'relu']])
model_fp32_fused

QM(
  (conv): ConvReLU2d(
    (0): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (relu): Identity()
  (quant): QuantStub()
  (dequant): DeQuantStub()
)

In [12]:
# 查看此时的数据类型
input_fp32 = torch.randn(4, 1, 4, 4)

x = model_fp32(input_fp32)
print('激活和权重的数据类型分别为：'f'{x.dtype}, {model_fp32.conv.weight.dtype}')

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32
激活和权重的数据类型分别为：torch.float32, torch.float32


3)  启用观测器

在融合后的模块中启用观测器，用于在校准期间观测激活(activation)张量。

In [13]:
model_fp32_prepared = torch.quantization.prepare(model_fp32_fused)

4)  校准准备好的模型

In [15]:
input_fp32 = torch.randn(4, 1, 4, 4)

x = model_fp32_prepared(input_fp32)
print('激活和权重的数据类型分别为：'f'{x.dtype}, {model_fp32.conv.weight.dtype}')

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32
激活和权重的数据类型分别为：torch.float32, torch.float32


5. 模型转换

量化权重，计算和存储每个激活张量要适用的尺度（scale)和偏差（bias)值， 并用量化实现替换关键算子

In [16]:
# 转换已经校准好的模型为量化模型
model_int8 = torch.quantization.convert(model_fp32_prepared)
model_int8

QM(
  (conv): QuantizedConvReLU2d(1, 1, kernel_size=(1, 1), stride=(1, 1), scale=0.011344349011778831, zero_point=0)
  (relu): Identity()
  (quant): Quantize(scale=tensor([0.0417]), zero_point=tensor([75]), dtype=torch.quint8)
  (dequant): DeQuantize()
)

In [17]:
# 查看权重的数据类型
model_int8.conv.weight().dtype

torch.qint8

In [18]:
# 查看权重的元素大小
model_int8.conv.weight().element_size()

1

In [19]:
# 运行模型，查看相关计算。相关计算将会在{data} torch.qint8中发生
res = model_int8(input_fp32)
res.dtype

量化前的类型： torch.quint8
量化中的类型： torch.quint8
量化后的类型： torch.float32


torch.float32